# Practical SQL: Chapter Practice Notebook

This Jupyter notebook will display all SQL queries and practices covered in each chapter of the book "Practical SQL: A Beginner's Guide To Storytelling With Data" by Anthony DeBarros.

The first part of this notebook will require us to import a couple of dependencies as well as personal data that will allow us to link out Jupyter Notebook to PostgreSQL.

In [1]:
import psycopg2
import pandas as pd
from sql_data import db, usr, pwd

In [2]:
# Connecting to postgreSQL database
conn = psycopg2.connect(
    host = "localhost",
    database = db,
    user = usr,
    password = pwd,
    port = 5432
)

def execute_query(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        column_names = [i[0] for i in cursor.description]
        results = pd.DataFrame(results, columns= column_names)
        return results
        return print("Query executed succesfully!")
        # Closing the cursor
        cursor.close()
        del cursor
        # Closing the connection
        connection.close()
    except OperationalError as e:
        print(f"The error '{e}' occurred.")

## Chapter 8: Extracting Information By Grouping and Summarizing

This chapter requires us to build two tables based on the 2009 and 2014 Public Library Surveys conducted by the Institute of Museum and Library Services (IMLS). Please refer to the books resources as detailed in the readme page to gain access to the csv files needed to fill in the two library tables used henceforth.

In [3]:
# Exploring the Library Data Using Aggregate Functions
# Counting Rows and Values using Count()

query = """
SELECT COUNT(*)
FROM pls_fy2014_pupld14a;
"""

execute_query(conn, query)

,count
0,9305


In [4]:
query = """
SELECT COUNT(*)
FROM pls_fy2009_pupld09a;
"""

execute_query(conn, query)

,count
0,9299


In [5]:
# Examining the count of rows in columns where the NOT NULL constraint was not applied
query = """
SELECT COUNT(salaries)
FROM pls_fy2014_pupld14a;
"""
execute_query(conn,query)

,count
0,5983


In [6]:
# Using the DISTINCT function to see how many unique values are contained within a column
# This query returns the count of all rows
query = """
SELECT COUNT(libname)
FROM pls_fy2014_pupld14a;
"""
execute_query(conn,query)

,count
0,9305


In [7]:
# This query return the unique values of the same column. The number should be smaller as there are some duplicated values.
query = """
SELECT COUNT(DISTINCT(libname))
FROM pls_fy2014_pupld14a;
"""
execute_query(conn,query)

,count
0,8515


In [8]:
# Finding Maximum and Minimum Values Using MAX() and MIN()
query = """
SELECT MAX(visits), MIN(visits)
FROM pls_fy2014_pupld14a;
"""
execute_query(conn, query)

,max,min
0,17729020,-3


In [9]:
# Aggregating Data Using GROUP BY

# Combining GROUP BY clause with COUNT()
query = """
SELECT stabr, COUNT(*)
FROM pls_fy2014_pupld14a
GROUP BY stabr
ORDER BY COUNT(*) DESC
LIMIT 5;
"""
execute_query(conn, query)

,stabr,count
0,NY,756
1,IL,625
2,TX,556
3,IA,543
4,PA,455


In [10]:
# Aggregating data from multiple columns using GROUP BY
query = """
SELECT stabr, stataddr, COUNT(*)
FROM pls_fy2014_pupld14a
GROUP BY stabr, stataddr
ORDER BY stabr ASC, COUNT(*) DESC;
"""

execute_query(conn, query)

,stabr,stataddr,count
0,AK,00,70
1,AK,15,10
2,AK,07,5
3,AL,00,221
4,AL,07,3
...,...,...,...
101,WI,07,6
102,WI,15,3
103,WV,00,93
104,WV,15,4


In [11]:
# Aggregating data from multiple joined tables

# No aggregation but here we are trying to determine trends in library visists using the 2014 and 2009 tables 
query = """
SELECT SUM(pls14.visits) AS visits_2014,
    SUM(pls09.visits) AS visits_2009
FROM pls_fy2014_pupld14a AS pls14
JOIN pls_fy2009_pupld09a AS pls09
    ON pls14.fscskey = pls09.fscskey
WHERE pls14.visits >=0 AND pls09.visits >= 0;
"""

execute_query(conn, query)

,visits_2014,visits_2009
0,1417299241,1585455205


In [12]:
# Aggregating data to compare trends by states
query = """
SELECT
    pls14.stabr,
    SUM(pls14.visits) AS visits_2014,
    SUM(pls09.visits) AS visits_2009,
    ROUND( (CAST(SUM(pls14.visits) AS DECIMAL(10, 1)) - SUM(pls09.visits)) /
        SUM(pls09.visits) * 100, 2) AS pct_change
FROM pls_fy2014_pupld14a AS pls14
JOIN pls_fy2009_pupld09a AS pls09
    ON pls14.fscskey = pls09.fscskey
WHERE pls14.visits >=0 AND pls09.visits >= 0
GROUP BY pls14.stabr
ORDER BY pct_change DESC
LIMIT 5
;
"""

execute_query(conn, query)

,stabr,visits_2014,visits_2009,pct_change
0,GU,103593,60763,70.49
1,DC,4230790,2944774,43.67
2,LA,17242110,15591805,10.58
3,MT,4582604,4386504,4.47
4,AL,17113602,16933967,1.06


In [13]:
# Filtering an Aggregate Query Using Having
query = """
SELECT
    pls14.stabr,
    SUM(pls14.visits) AS visits_2014,
    SUM(pls09.visits) AS visits_2009,
    ROUND( (CAST(SUM(pls14.visits) AS DECIMAL(10, 1)) - SUM(pls09.visits)) /
        SUM(pls09.visits) * 100, 2) AS pct_change
FROM pls_fy2014_pupld14a AS pls14
JOIN pls_fy2009_pupld09a AS pls09
    ON pls14.fscskey = pls09.fscskey
WHERE pls14.visits >=0 AND pls09.visits >= 0
GROUP BY pls14.stabr
HAVING SUM(pls14.visits) > 50000000
ORDER BY pct_change DESC
LIMIT 5
;
"""

execute_query(conn, query)

,stabr,visits_2014,visits_2009,pct_change
0,TX,72876601,78838400,-7.56
1,CA,162787836,182181408,-10.65
2,OH,82495138,92402369,-10.72
3,NY,106453546,119810969,-11.15
4,IL,72598213,82438755,-11.94


## Chapter 9: Inspecting and Modifying Data

For this chaper, we will use a direcory of U.S. meat, poultry, and egg producers provided by the Food Safety and Inspection Service (FSIS)  agency within the U.S. Department of Agriculture.

These first initial queries should give us some insight into our dataset:

In [14]:
# Reviweing our imported data
query = """
SELECT COUNT(*)
FROM meat_poultry_egg_inspect;
"""

execute_query(conn, query)

,count
0,6201


In [15]:
# Looking for duplicates
query = """
SELECT company,
    street,
    city,
    st,
    COUNT(*) AS address_count
FROM meat_poultry_egg_inspect
GROUP BY company, street, city, st
HAVING COUNT(*) > 1
ORDER BY company, street, city, st;
"""

execute_query(conn, query)

,company,street,city,st,address_count
0,Acre Station Meat Farm,17076 Hwy 32 N,Pinetown,NC,2
1,Beltex Corporation,3801 North Grove Street,Fort Worth,TX,2
2,Cloverleaf Cold Storage,111 Imperial Drive,Sanford,NC,2
3,"Crete Core Ingredients, LLC",2220 County Road I,Crete,NE,2
4,"Crider, Inc.",1 Plant Avenue,Stillmore,GA,3
5,"Dimension Marketing & Sales, Inc.",386 West 9400 South,Sandy,UT,2
6,"Foster Poultry Farms, A California Corporation",6648 Highway 15 North,Farmerville,LA,2
7,"Freezer & Dry Storage, LLC",21740 Trolley Industrial Drive,Taylor,MI,2
8,JBS Souderton Inc.,249 Allentown Road,Souderton,PA,2
9,KB Poultry Processing LLC,15024 Sandstone Dr.,Utica,MN,2


In [16]:
# Checking for missing values
query = """
SELECT st,
    COUNT(*) AS state_count
FROM meat_poultry_egg_inspect
GROUP BY st
ORDER BY st NULLS FIRST
LIMIT 10
"""

execute_query(conn, query)

,st,state_count
0,AK,17
1,AL,94
2,AR,87
3,AS,1
4,AZ,37
5,CA,666
6,CO,121
7,CT,55
8,DC,2
9,DE,22


In [17]:
# Looking at rows that don't have a value in a column(s)
query = """
SELECT est_number,
    company,
    city,
    st,
    zip
FROM meat_poultry_egg_inspect
WHERE st IS NULL;
"""

execute_query(conn, query)

,est_number,company,city,st,zip


In [18]:
# Checking for inconsitent data values
query = """
SELECT company,
    COUNT(*)
FROM meat_poultry_egg_inspect
GROUP BY company
ORDER BY company ASC;
"""

execute_query(conn, query)

,company,count
0,121 In-Flight Catering LLC,1
1,165368 C. Corporation,1
2,1732 Meats LLC,1
3,"1st Original Texas Chili Company, Inc.",1
4,290 West Bar & Grill,1
...,...,...
5497,"Zrile Bros. Packing Co.,Inc",1
5498,"Zummo Meat Co., Inc.",1
5499,"Zwanenberg Food Group (USA), Inc.",1
5500,Zweigle's Inc.,1


In [19]:
# Checking for malformed values using length() - here we see that because the zip column was read in as a number rather than as text it lost the leading 0's
query = """
SELECT LENGTH(zip),
    COUNT(*) AS length_count
FROM meat_poultry_egg_inspect
GROUP BY LENGTH(zip)
ORDER BY LENGTH(zip);
"""

execute_query(conn, query)

,length,length_count
0,5,6201


In [20]:
# Examining rows that lost leading 0's
query = """
SELECT st,
    COUNT(*) AS st_count
FROM meat_poultry_egg_inspect
WHERE LENGTH(zip) < 5
GROUP BY st
ORDER BY st;
"""

execute_query(conn, query)

,st,st_count


For the following part of the chapter an identical copy of the st column was created in the meat_poultry_egg_inspect table titled st_copy

In [21]:
# Modifying Tables, Columns, and Data
# Updating the rows where the values were missing
query = """
UPDATE meat_poultry_egg_inspect
SET st = 'MN'
WHERE est_number = 'V18677A';

UPDATE meat_poultry_egg_inspect
SET st = 'AL'
WHERE est_number = 'M45319+P45319';

UPDATE meat_poultry_egg_inspect
SET st = 'WI'
WHERE est_number = 'M263A+P263A+V263A';

SELECT est_number,
    company,
    city,
    st,
    zip
FROM meat_poultry_egg_inspect
WHERE st IS NULL; 
"""

# The last part of the query just inspects the table to find NULL values - after our update there should be none
execute_query(conn, query)


,est_number,company,city,st,zip


For the following part of the chapter, an identical copy of the company column was created in the meat_poultry_egg_inspect table titled company_standard

In [22]:
# Updating values for consistency
query = """
UPDATE meat_poultry_egg_inspect
SET company_standard = company;

SELECT * FROM meat_poultry_egg_inspect LIMIT 5;
"""

execute_query(conn, query)

,est_number,company,street,city,st,zip,phone,grant_date,activities,dbas,st_copy,company_standard,inspection_date,reviewed_date,meat_processing,poultry_processing
0,M263A+P263A+V263A,Jones Dairy Farm,801 Rockwell Ave.,None,WI,53538,(920) 563-2431,2016-06-22,"Identification - Meat, Identification - Poultr...",Greensboro Packers; Habbersett; Ralph & Paul A...,None,Jones Dairy Farm,None,2018-02-07,True,True
1,V3507,Zollinger Cold Storage Corporation,699 West 1700 South,Logan,UT,84321,(801) 753-1134,1990-02-27,"Certification - Export, Identification - Meat,...",None,UT,Zollinger Cold Storage Corporation,None,2018-02-07,None,None
2,V971,Supervalu Inc.,1400 West Gadsden Street,Quincy,FL,32351,(850) 875-2600,2011-09-16,"Certification - Export, Identification - Meat,...",None,FL,Supervalu Inc.,None,2018-02-07,None,None
3,M1544,J & M Meat Co.,545 9th St.,Oakland,CA,94607,(510) 839-0380,1982-05-24,Meat Processing,None,CA,J & M Meat Co.,None,2018-02-07,True,None
4,M34258,Beatrice Meat Company LLC,2595 Grant Blvd.,Beatrice,AL,36425,(251) 575-4021,2007-08-23,Meat Processing,None,AL,Beatrice Meat Company LLC,None,2018-02-07,True,None


In [23]:
# Updating and standardizing all rows that contain the string 'Armour' to appear in company_standard as Armour_Eckrich Meats
query = """
UPDATE meat_poultry_egg_inspect
SET company_standard = 'Armour_Eckrich Meats'
WHERE company LIKE '%Armour%';

SELECT company, company_standard
FROM meat_poultry_egg_inspect
WHERE company LIKE '%Armour%';
"""

execute_query(conn, query)

,company,company_standard
0,Armour-Eckrich Meats LLC,Armour_Eckrich Meats
1,Armour-Eckrich Meats LLC,Armour_Eckrich Meats
2,"Armour-Eckrich Meats, Inc.",Armour_Eckrich Meats
3,"Armour-Eckrich Meats, LLC",Armour_Eckrich Meats
4,"Armour-Eckrich Meats, LLC",Armour_Eckrich Meats
5,"Armour - Eckrich Meats, LLC",Armour_Eckrich Meats
6,Armour-Eckrich Meats LLC,Armour_Eckrich Meats


For the following part of the chapter, an identicalcopy of the zip column was created in the meat_poultry_egg_inspect table titled zip_copy

In [24]:
query = """
UPDATE meat_poultry_egg_inspect
SET zip_copy = zip;

SELECT * FROM meat_poultry_egg_inspect LIMIT 5;
"""

execute_query(conn, query)

NameError: name 'OperationalError' is not defined

In [25]:
# Removing ZIP codes using concatenation
query = """
UPDATE meat_poultry_egg_inspect
SET zip = '00' || zip
WHERE st IN('PR', 'VI') AND LENGTH(zip) = 3;

SELECT * FROM meat_poultry_egg_inspect LIMIT 5;
"""

execute_query(conn, query)

,est_number,company,street,city,st,zip,phone,grant_date,activities,dbas,st_copy,company_standard,zip_copy
0,M6714+P6714,A. Concepcion Hnos,105 Aguila Street,Mayaguez,PR,00680,(809) 832-4145,1993-02-09,"Meat Processing, Poultry Processing",None,PR,A. Concepcion Hnos,00680
1,M17820+P17820+V17820,Sunset Foods,Sunset Foods 1640 Fuller Road,West Des Moines,IA,50265,(515) 222-4477,2007-05-23,"Meat Processing, Poultry Processing, Voluntary...",None,IA,Sunset Foods,50265
2,M9330,Younis Inc.,1124 Howell Street,Seattle,WA,98101,(206) 624-9248,2016-08-04,Meat Processing,Market House,WA,Younis Inc.,98101
3,M45020+P45020,"Yuris Food, LTD","6933 Flintlock Road, Bldg. #4",Houston,TX,77040,(713) 547-4601,2014-08-21,"Meat Processing, Poultry Processing",None,TX,"Yuris Food, LTD",77040
4,M10249+P10249,"Zalack's Flint Provision, Inc.",2801 Lippincott Blvd,Flint,MI,48507,(810) 742-4100,2004-04-26,"Meat Processing, Poultry Processing","Flint Provision, Inc.; Zalack's Provision, Inc.",MI,"Zalack's Flint Provision, Inc.",48507


In [26]:
# Updating the remaining rows lacking leading zero's
query = """
UPDATE meat_poultry_egg_inspect
SET zip = '0' || zip
WHERE st IN('CT', 'MA', 'ME', 'NH', 'NJ', 'RI', 'VT') AND LENGTH(zip) = 4;

SELECT * FROM meat_poultry_egg_inspect LIMIT 5;
"""

execute_query(conn, query)

,est_number,company,street,city,st,zip,phone,grant_date,activities,dbas,st_copy,company_standard,zip_copy
0,M6714+P6714,A. Concepcion Hnos,105 Aguila Street,Mayaguez,PR,00680,(809) 832-4145,1993-02-09,"Meat Processing, Poultry Processing",None,PR,A. Concepcion Hnos,00680
1,M17820+P17820+V17820,Sunset Foods,Sunset Foods 1640 Fuller Road,West Des Moines,IA,50265,(515) 222-4477,2007-05-23,"Meat Processing, Poultry Processing, Voluntary...",None,IA,Sunset Foods,50265
2,M9330,Younis Inc.,1124 Howell Street,Seattle,WA,98101,(206) 624-9248,2016-08-04,Meat Processing,Market House,WA,Younis Inc.,98101
3,M45020+P45020,"Yuris Food, LTD","6933 Flintlock Road, Bldg. #4",Houston,TX,77040,(713) 547-4601,2014-08-21,"Meat Processing, Poultry Processing",None,TX,"Yuris Food, LTD",77040
4,M10249+P10249,"Zalack's Flint Provision, Inc.",2801 Lippincott Blvd,Flint,MI,48507,(810) 742-4100,2004-04-26,"Meat Processing, Poultry Processing","Flint Provision, Inc.; Zalack's Provision, Inc.",MI,"Zalack's Flint Provision, Inc.",48507


In [27]:
# Making sure that we fixed all zip codes
query = """
SELECT LENGTH(zip),
    COUNT(zip)
FROM meat_poultry_egg_inspect
GROUP BY LENGTH(zip);
"""

execute_query(conn, query)

,length,count
0,5,6287


For the following part of the chapter, a table was created in order to se tan inspection date for each company by U.S. regions

In [38]:
query = """
UPDATE meat_poultry_egg_inspect AS inspect
SET inspection_date = '2019-12-01'
WHERE EXISTS (SELECT state_regions.region
        FROM state_regions
        WHERE inspect.st = state_regions.st
            AND state_regions.region = 'New England');

SELECT st, inspection_date
FROM meat_poultry_egg_inspect
GROUP BY st, inspection_date
ORDER BY st
LIMIT 10;
"""

execute_query(conn, query)

,st,inspection_date
0,AK,None
1,AL,None
2,AR,None
3,AS,None
4,AZ,None
5,CA,None
6,CO,None
7,CT,2019-12-01
8,DC,None
9,DE,None


Deleting Data

In [40]:
# Using DELETE FROM and WHERE to delete all companies in U.S. territories
query = """
DELETE FROM meat_poultry_egg_inspect
WHERE st IN('PR', 'VI');

SELECT st
FROM meat_poultry_egg_inspect
WHERE st IN ('PR', 'VI');
"""

execute_query(conn, query)

,st


In [41]:
# Deleting a column from a table - here we drop the zip_copy column from the meat_poultry_egg_inspect table
query = """
ALTER TABLE meat_poultry_egg_inspect
DROP COLUMN zip_copy;

SELECT * FROM meat_poultry_egg_inspect;
"""

execute_query(conn, query)

,est_number,company,street,city,st,zip,phone,grant_date,activities,dbas,st_copy,company_standard,inspection_date
0,M17820+P17820+V17820,Sunset Foods,Sunset Foods 1640 Fuller Road,West Des Moines,IA,50265,(515) 222-4477,2007-05-23,"Meat Processing, Poultry Processing, Voluntary...",None,IA,Sunset Foods,None
1,M9330,Younis Inc.,1124 Howell Street,Seattle,WA,98101,(206) 624-9248,2016-08-04,Meat Processing,Market House,WA,Younis Inc.,None
2,M45743+P45743,5th Generation Adams Farm,15 Higley Adams Road,Wilmington,VT,05363,(802) 464-3762,2016-03-03,"Meat Processing, Meat Slaughter, Poultry Proce...",Adams Farm,VT,5th Generation Adams Farm,2019-12-01
3,M45020+P45020,"Yuris Food, LTD","6933 Flintlock Road, Bldg. #4",Houston,TX,77040,(713) 547-4601,2014-08-21,"Meat Processing, Poultry Processing",None,TX,"Yuris Food, LTD",None
4,M10249+P10249,"Zalack's Flint Provision, Inc.",2801 Lippincott Blvd,Flint,MI,48507,(810) 742-4100,2004-04-26,"Meat Processing, Poultry Processing","Flint Provision, Inc.; Zalack's Provision, Inc.",MI,"Zalack's Flint Provision, Inc.",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,M6863+P6863,"Armour-Eckrich Meats, LLC","3311 South, Hwy 19 State Road 19 South",Peru,IN,46970,(765) 473-3086,2015-11-03,"Meat Processing, Poultry Processing",Agar Foods; Ark Valley; Armour Food Company; A...,IN,Armour_Eckrich Meats,None
6197,M276+P276,"Advance Pierre Foods, Inc.",70 Saint John Street,Portland,ME,04102,(207) 541-2800,2015-06-10,"Meat Processing, Poultry Processing",Advance Brands LLC; Barber Foods; Chef's Pantr...,ME,"Advance Pierre Foods, Inc.",2019-12-01
6198,M40161+P40161,"Afya Brands International, Inc.",2781 US Route 5 North,Windsor,VT,05089,(603) 676-8591,2016-01-28,"Meat Processing, Poultry Processing",Global Village Cuisine,VT,"Afya Brands International, Inc.",2019-12-01
6199,M45230,Against The Grain Gourmet,22 Browne Court Unit #119,Brattleboro,VT,05301,(802) 258-3838,2014-10-15,Meat Processing,None,VT,Against The Grain Gourmet,2019-12-01


Using Transaction Blocks to save or revert changes

The following queries would be executed to show how we can make changes to a table without fully committing them - thus giving us the opportunity to revert back to a point prior to our alteration of data

        START TRANSACTION;

        UPDATE meat_poultry_egg_inspect
        SET company = 'AGRO Merchants Oakland LLC'
        WHERE company = 'AGRO Merchants Oakland, LLC';

        SELECT company
        FROM meat_poultry_egg_inspect
        WHERE company LIKE 'AGRO%'
        ORDER BY company
        LIMIT 5;

        ROLLBACK;


For the following part of this chapter, a new table called meat_poultry_egg_inspect_backup was created to demonstrate how to improve performance when updating tables. Please refer to the book documentation for more details.

In the following query we create a new meat_poultry_egg_inspect table with a new column to avoid taking up additional space that would come with just adding a column


    CREATE TABLE meat_poultry_egg_inspect_backup AS
    SELECT *,
        '2018-02-07'::date AS reviewed_date
    FROM meat_poultry_egg_inspect;


After the table is created, we can then use the following query to swap this newly created table's name with the original:

    ALTER TABLE meat_poultry_egg_inspect RENAME TO meat_poultry_egg_inspect_temp;
    ALTER TABLE meat_poultry_egg_inspect_backup RENAME TO meat_poultry_egg_inspect;
    ALTER TABLE meat_poultry_egg_inspect_temp RENAME TO meat_poultry_egg_backup;

In [31]:
# Ensuring that our tables have switched names - by doing this process we avoid updating rows and having the db inflate the size of the table. Eventually the "backup" table will be dropped and the remaning data table will be smaller
query = """
SELECT * FROM meat_poultry_egg_inspect LIMIT 5;
"""

execute_query(conn, query)

,est_number,company,street,city,st,zip,phone,grant_date,activities,dbas,st_copy,company_standard,inspection_date,reviewed_date
0,M45461+P45461,Zeera India Catering Co.,1116 S. Rainbow Blvd.,Las Vegas,NV,89146,(702) 538-2945,2016-02-11,Poultry Processing,Z India,NV,Zeera India Catering Co.,None,2018-02-07
1,M9776,Zeiler-Caruth Quality Meats,139 Callapoose Road,Moscow,PA,18444,(570) 689-2350,2010-10-06,"Meat Processing, Meat Slaughter",None,PA,Zeiler-Caruth Quality Meats,None,2018-02-07
2,M6308+P6308,"Zenner's Quality Meat Products, Inc.",2131 NW Kearney Street,Portland,OR,97210,(503) 241-4113,2015-05-27,"Meat Processing, Poultry Processing",Zenner's Sausage Company,OR,"Zenner's Quality Meat Products, Inc.",None,2018-02-07
3,V3507,Zollinger Cold Storage Corporation,699 West 1700 South,Logan,UT,84321,(801) 753-1134,1990-02-27,"Certification - Export, Identification - Meat,...",None,UT,Zollinger Cold Storage Corporation,None,2018-02-07
4,M21797+P21797,Zook's Homemade Chicken Pies LLC,3194 Harvest Drive,Ronks,PA,17572,(717) 768-0239,2010-09-13,"Meat Processing, Poultry Processing",None,PA,Zook's Homemade Chicken Pies LLC,None,2018-02-07


## Chapter 10: Statistical Functions in SQL

For this chapter, a data table was created from the 2011-2015 American Community Survey. Please see the book documentation for more information.

In [25]:
# Examining our newly created ACS 5 year estimate table
query = "SELECT * FROM acs_2011_2015_stats;"

execute_query(conn, query)

,geoid,county,st,pct_travel_60_min,pct_bachelors_higher,pct_masters_higher,median_hh_income
0,0500000US01001,Autauga County,Alabama,5.630,23.230,9.747,51281.0
1,0500000US01003,Baldwin County,Alabama,7.966,28.985,9.724,50254.0
2,0500000US01005,Barbour County,Alabama,7.672,12.516,5.009,32964.0
3,0500000US01007,Bibb County,Alabama,9.198,10.647,4.594,38678.0
4,0500000US01009,Blount County,Alabama,16.741,12.891,4.273,45813.0
...,...,...,...,...,...,...,...
3137,0500000US56037,Sweetwater County,Wyoming,6.741,19.599,5.810,69022.0
3138,0500000US56039,Teton County,Wyoming,1.720,53.932,16.115,75325.0
3139,0500000US56041,Uinta County,Wyoming,8.460,19.306,5.373,56569.0
3140,0500000US56043,Washakie County,Wyoming,2.763,21.131,6.121,47652.0


In [26]:
# Measuring Correlations with CORR(Y, X) - Here we examine the correlation between the median_hh_income and pct_bachelors_higher columns
query = """
SELECT CORR(median_hh_income, pct_bachelors_higher) AS bachelors_incomre_r
FROM acs_2011_2015_stats;
"""

execute_query(conn, query)

,bachelors_incomre_r
0,0.682186


In [27]:
# Checking addditional correlation coefficients
query = """
SELECT 
    ROUND(
        CORR(median_hh_income, pct_bachelors_higher)::numeric, 2
    ) AS bachelors_income_r,
    ROUND(
        CORR(pct_travel_60_min, median_hh_income)::numeric, 2
    ) AS income_travel_r,
    ROUND(
        CORR(pct_travel_60_min, pct_bachelors_higher)::numeric, 2
    ) AS bachelors_travel_r
FROM acs_2011_2015_stats;
"""

execute_query(conn, query)

,bachelors_income_r,income_travel_r,bachelors_travel_r
0,0.68,0.05,-0.14


In [28]:
# Predicting Values with Regression Analysis - here we try to predict what the expected median household income in a county would be if the percntage of people with a bachelor's degree or higher in that county was 30 percent. The first step is to try and find what our slope and intercepts will be using the REGR_SLOPE(Y, X) AND REGR_INTERCEPT(Y, X) functions
query = """
SELECT 
    ROUND(
        REGR_SLOPE(median_hh_income, pct_bachelors_higher)::numeric, 2
    ) AS slope,
    ROUND(
        REGR_INTERCEPT(median_hh_income, pct_bachelors_higher)::numeric, 2
    ) AS y_intercept
FROM acs_2011_2015_stats;
"""

execute_query(conn, query)

,slope,y_intercept
0,926.95,27901.15


In [30]:
# Using this info we can calculate what the median household income would be using our slope-intercept formula
y = (926.95*30)+27901.15
print(y)

55709.65
